
# Anchors: High-Precision Model-Agnostic Explanations

Paper Link: https://homes.cs.washington.edu/~marcotcr/aaai18.pdf




In this tutorial, we will explore how Explainable AI methods like ANCHOR can help explain the predictions made by a text classifier.



### Installation and Imports

In [ ]:
!pip install spacy && python -m spacy download en_core_web_sm
!pip install torch transformers spacy && python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 29.5 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 28.8 MB/s 
     |████████████████████████████████| 6.6 MB 50.9 MB/s 
     |████████████████████████████████| 120 kB 71.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.4.0/en_core_web_sm-3.4.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
!pip install anchor-exp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 427 kB 16.4 MB/s 
     |████████████████████████████████| 275 kB 56.4 MB/s 
  Created wheel for anchor-exp: filename=anchor_exp-0.0.2.0-py3-none-any.whl size=433521 sha256=6c7044cf24d2f170d67252d1dc389f9e808891c6b6d45bf744e032af7ead7eca
  Stored in directory: /root/.cache/pip/wheels/ef/70/33/efea781f1c5de2489d9da29b1cfd6053a855423ffbe904a3bc
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283857 sha256=eefd60888880fce848cc642c08452f47cc97958895bac5728a95e130ed1bf9df
  Stored in directory: /root/.cache/pip/wheels/ca/cb/e5/ac701e12d365a08917bf4c6171c0961bc880a8181359c66aa7
Successfully built anchor-exp lime


In [ ]:
%load_ext autoreload
%autoreload 2
import os
import os.path
import numpy as np
import sklearn
import sklearn.model_selection
import sklearn.linear_model
import sklearn.ensemble
import spacy
import sys
from sklearn.feature_extraction.text import CountVectorizer
from anchor import anchor_text
import time

## Text Classification Task

<style>
#output-body {
    display: flex;
    align-items: center;
    justify-content: center;
}
</style>
<img src=https://cdn-icons-png.flaticon.com/512/415/415682.png width="100" 
     height="100" align=left> <img src=https://cdn-icons-png.flaticon.com/512/5219/5219604.png width=90 height=90 align=left> <img src=https://cdn-icons-png.flaticon.com/512/2933/2933245.png width=100 height=100 align=left>


Our job in this tutorial is to classify sentences into two categories, Apple Fruit and Apple Computers. 



In [ ]:
apple_fruit_wiki = """An apple is an edible fruit produced by an apple tree (Malus domestica). Apple trees are cultivated worldwide and are the most widely grown species in the genus Malus. The tree originated in Central Asia, where its wild ancestor, Malus sieversii, is still found today. Apples have been grown for thousands of years in Asia and Europe and were brought to North America by European colonists. Apples have religious and mythological significance in many cultures, including Norse, Greek, and European Christian tradition.

Apples grown from seed tend to be very different from those of their parents, and the resultant fruit frequently lacks desired characteristics. Generally, apple cultivars are propagated by clonal grafting onto rootstocks. Apple trees grown without rootstocks tend to be larger and much slower to fruit after planting. Rootstocks are used to control the speed of growth and the size of the resulting tree, allowing for easier harvesting.

There are more than 7,500 known cultivars of apples. Different cultivars are bred for various tastes and uses, including cooking, eating raw, and cider production. Trees and fruit are prone to a number of fungal, bacterial, and pest problems, which can be controlled by a number of organic and non-organic means. In 2010, the fruit's genome was sequenced as part of research on disease control and selective breeding in apple production.

Worldwide production of apples in 2018 was 86 million tonnes, with China accounting for nearly half of the total.[3] The word apple, formerly spelled æppel in Old English, is derived from the Proto-Germanic root *ap(a)laz, which could also mean fruit in general. This is ultimately derived from Proto-Indo-European *ab(e)l-, but the precise original meaning and the relationship between both words[clarification needed] is uncertain.

As late as the 17th century, the word also functioned as a generic term for all fruit other than berries but including nuts—such as the 14th century Middle English word appel of paradis, meaning a banana.[4] This use is analogous to the French language use of pomme. Blossoms are produced in spring simultaneously with the budding of the leaves and are produced on spurs and some long shoots. The 3 to 4 cm (1 to 1+1⁄2 in) flowers are white with a pink tinge that gradually fades, five petaled, with an inflorescence consisting of a cyme with 4–6 flowers. The central flower of the inflorescence is called the "king bloom"; it opens first and can develop a larger fruit.[5][6]

The fruit is a pome that matures in late summer or autumn, and cultivars exist in a wide range of sizes. Commercial growers aim to produce an apple that is 7 to 8.5 cm (2+3⁄4 to 3+1⁄4 in) in diameter, due to market preference. Some consumers, especially those in Japan, prefer a larger apple, while apples below 5.5 cm (2+1⁄4 in) are generally used for making juice and have little fresh market value. The skin of ripe apples is generally red, yellow, green, pink, or russetted, though many bi- or tri-colored cultivars may be found.[7] The skin may also be wholly or partly russeted i.e. rough and brown. The skin is covered in a protective layer of epicuticular wax.[8] The exocarp (flesh) is generally pale yellowish-white,[7] though pink, yellow or green exocarps also occur. Apples are diploid (though triploid cultivars are not uncommon), have 17 chromosomes and an estimated genome size of approximately 650 Mb. Several whole genome sequences have been made available, the first one in 2010 was based on the diploid cultivar 'Golden Delicious'.[14] However, this first whole genome sequence turned out to contain several errors[15] in part owing to the high degree of heterozygosity in diploid apples which, in combination with an ancient genome duplication, complicated the assembly. Recently, double- and trihaploid individuals have been sequenced, yielding whole genome sequences of higher quality.[16][17] The first whole genome assembly was estimated to contain around 57,000 genes,[14] though the more recent genome sequences support more moderate estimates between 42,000 and 44,700 protein-coding genes.[16][17] Among other things, the availability of whole genome sequences has provided evidence that the wild ancestor of the cultivated apple most likely is Malus sieversii. Re-sequencing of multiple accessions has supported this, while also suggesting extensive introgression from Malus sylvestris following domestication.[18] Malus sieversii is recognized as a major progenitor species to the cultivated apple, and is morphologically similar. Due to the genetic variability in Central Asia, this region is generally considered the center of origin for apples.[19] The apple is thought to have been domesticated 4000–10000 years ago in the Tian Shan mountains, and then to have travelled along the Silk Road to Europe, with hybridization and introgression of wild crabapples from Siberia (M. baccata), the Caucasus (M. orientalis), and Europe (M. sylvestris). Only the M. sieversii trees growing on the western side of the Tian Shan mountains contributed genetically to the domesticated apple, not the isolated population on the eastern side.[18]

Chinese soft apples, such as M. asiatica and M. prunifolia, have been cultivated as dessert apples for more than 2000 years in China. These are thought to be hybrids between M. baccata and M. sieversii in Kazakhstan.[18]

Among the traits selected for by human growers are size, fruit acidity, color, firmness, and soluble sugar. Unusually for domesticated fruits, the wild M. sieversii origin is only slightly smaller than the modern domesticated apple.[18]

At the Sammardenchia-Cueis site near Udine in Northeastern Italy, seeds from some form of apples have been found in material carbon dated to around 4000 BCE.[20] Genetic analysis has not yet been successfully used to determine whether such ancient apples were wild Malus sylvestris or Malus domesticus containing Malus sieversii ancestry.[21] It is generally also hard to distinguish in the archeological record between foraged wild apples and apple plantations.

There is indirect evidence of apple cultivation in the third millennium BCE in the Middle East. There was substantial apple production in the European classical antiquity, and grafting was certainly known then.[21] Grafting is an essential part of modern domesticated apple production, to be able to propagate the best cultivars; it is unclear when apple tree grafting was invented.[21]

Winter apples, picked in late autumn and stored just above freezing, have been an important food in Asia and Europe for millennia.[22] Of the many Old World plants that the Spanish introduced to Chiloé Archipelago in the 16th century, apple trees became particularly well adapted.[23] Apples were introduced to North America by colonists in the 17th century,[5] and the first apple orchard on the North American continent was planted in Boston by Reverend William Blaxton in 1625.[24] The only apples native to North America are crab apples, which were once called "common apples".[25] Apple cultivars brought as seed from Europe were spread along Native American trade routes, as well as being cultivated on colonial farms. An 1845 United States apples nursery catalogue sold 350 of the "best" cultivars, showing the proliferation of new North American cultivars by the early 19th century.[25] In the 20th century, irrigation projects in Eastern Washington began and allowed the development of the multibillion-dollar fruit industry, of which the apple is the leading product.[5]

Until the 20th century, farmers stored apples in frostproof cellars during the winter for their own use or for sale. Improved transportation of fresh apples by train and road replaced the necessity for storage.[26][27] Controlled atmosphere facilities are used to keep apples fresh year-round. Controlled atmosphere facilities use high humidity, low oxygen, and controlled carbon dioxide levels to maintain fruit freshness. They were first used in the United States in the 1960s.[28] """


apple_company_wiki = """Apple Inc. is an American multinational technology company that specializes in consumer electronics, software and online services headquartered in Cupertino, California, United States. Apple is the largest technology company by revenue (totaling US$365.8 billion in 2021) and as of June 2022, is the world's biggest company by market capitalization, the fourth-largest personal computer vendor by unit sales and second-largest mobile phone manufacturer. It is one of the Big Five American information technology companies, alongside Alphabet, Amazon, Meta, and Microsoft.

Apple was founded as Apple Computer Company on April 1, 1976, by Steve Jobs, Steve Wozniak and Ronald Wayne to develop and sell Wozniak's Apple I personal computer. It was incorporated by Jobs and Wozniak as Apple Computer, Inc. in 1977 and the company's next computer, the Apple II, became a best seller and one of the first mass-produced microcomputers. Apple went public in 1980 to instant financial success. The company developed computers featuring innovative graphical user interfaces, including the 1984 original Macintosh, announced in a critically acclaimed advertisement directed by Ridley Scott. By 1985, the high cost of its products and power struggles between executives caused problems. Wozniak stepped back from Apple amicably and pursued other ventures, while Jobs resigned bitterly and founded NeXT, taking some Apple employees with him.

As the market for personal computers expanded and evolved throughout the 1990s, Apple lost considerable market share to the lower-priced duopoly of the Microsoft Windows operating system on Intel-powered PC clones (also known as "Wintel"). In 1997, weeks away from bankruptcy, the company bought NeXT to resolve Apple's unsuccessful operating system strategy and entice Jobs back to the company. Over the next decade, Jobs guided Apple back to profitability through a number of tactics including introducing the iMac, iPod, iPhone and iPad to critical acclaim, launching "Think different" and other memorable advertising campaigns, opening the Apple Store retail chain, and acquiring numerous companies to broaden the company's product portfolio. When Jobs resigned in 2011 for health reasons, and died two months later, he was succeeded as CEO by Tim Cook.

Apple became the first publicly traded U.S. company to be valued at over $1 trillion in August 2018, then $2 trillion in August 2020, and most recently $3 trillion in January 2022. The company receives criticism regarding the labor practices of its contractors, its environmental practices, and its business ethics, including anti-competitive practices and materials sourcing. Nevertheless, the company has a large following and enjoys a high level of brand loyalty. It is ranked as one of the world's most valuable brands. Apple Computer Company was founded on April 1, 1976, by Steve Jobs, Steve Wozniak, and Ronald Wayne as a business partnership.[7][10] The company's first product was the Apple I, a computer designed and hand-built entirely by Wozniak.[11][12] To finance its creation, Jobs sold his only motorized means of transportation, a VW Bus, for a few hundred dollars, and Wozniak sold his HP-65 calculator for US$500 (equivalent to $2,381 in 2021).[13] Wozniak debuted the first prototype Apple I at the Homebrew Computer Club in July 1976.[14][15] The Apple I was sold as a motherboard with CPU, RAM, and basic textual-video chips—a base kit concept which would not yet be marketed as a complete personal computer.[16] It went on sale soon after debut for US$666.66 (equivalent to $3,175 in 2021).[17][18][19][20][21]: 180  Wozniak later said he was unaware of the coincidental mark of the beast in the number 666, and that he came up with the price because he liked "repeating digits".[22]

Apple Computer, Inc. was incorporated on January 3, 1977,[23][24] without Wayne, who had left and sold his share of the company back to Jobs and Wozniak for $800 only twelve days after having co-founded Apple.[25][26] Multimillionaire Mike Markkula provided essential business expertise and funding of US$250,000 (equivalent to $1,117,930 in 2021) to Jobs and Wozniak during the incorporation of Apple.[27][28] During the first five years of operations, revenues grew exponentially, doubling about every four months. Between September 1977 and September 1980, yearly sales grew from $775,000 to $118 million, an average annual growth rate of 533%.[29][30]

The Apple II, also invented by Wozniak, was introduced on April 16, 1977, at the first West Coast Computer Faire.[31] It differed from its major rivals, the TRS-80 and Commodore PET, because of its character cell-based color graphics and open architecture. While the Apple I and early Apple II models used ordinary audio cassette tapes as storage devices, they were superseded by the introduction of a 5+1⁄4-inch floppy disk drive and interface called the Disk II in 1978.[32][33]

The Apple II was chosen to be the desktop platform for the first "killer application" of the business world: VisiCalc, a spreadsheet program released in 1979.[32] VisiCalc created a business market for the Apple II and gave home users an additional reason to buy an Apple II: compatibility with the office.[32] Before VisiCalc, Apple had been a distant third place competitor to Commodore and Tandy.[34][35] By the end of the 1970s, Apple had become the leading computer manufacturer in the United States.[36]

On December 12, 1980, Apple (ticker symbol "AAPL") went public selling 4.6 million shares at $22 per share ($.39 per share when adjusting for stock splits as of March 30, 2019),[24] generating over $100 million, which was more capital than any IPO since Ford Motor Company in 1956.[37] By the end of the day, 300 millionaires were created, from a stock price of $29 per share[38] and a market cap of $1.778 billion.[37][38] A critical moment in the company's history came in December 1979 when Jobs and several Apple employees, including human–computer interface expert Jef Raskin, visited Xerox PARC in to see a demonstration of the Xerox Alto, a computer using a graphical user interface. Xerox granted Apple engineers three days of access to the PARC facilities in return for the option to buy 100,000[39] shares (5.6 million split-adjusted shares as of March 30, 2019)[24] of Apple at the pre-IPO price of $10 a share.[40] After the demonstration, Jobs was immediately convinced that all future computers would use a graphical user interface, and development of a GUI began for the Apple Lisa, named after Jobs's daughter.[41][42]

The Lisa division would be plagued by infighting, and in 1982 Jobs was pushed off the project. The Lisa launched in 1983 and became the first personal computer sold to the public with a GUI, but was a commercial failure due to its high price and limited software titles.[43]

Jobs, angered by being pushed off the Lisa team, took over the company's Macintosh division. Wozniak and Raskin had envisioned the Macintosh as low-cost-computer with a text-based interface like the Apple II, but a plane crash in 1981 forced Wozniak to step back from the project. Jobs quickly redefined the Macintosh as a graphical system that would be cheaper than the Lisa, undercutting his former division.[44] Jobs was also hostile to the Apple II division, which at the time, generated most of the company's revenue.[45]

In 1984, Apple launched the Macintosh, the first personal computer to be sold without a programming language.[46] Its debut was signified by "1984", a $1.5 million television advertisement directed by Ridley Scott that aired during the third quarter of Super Bowl XVIII on January 22, 1984.[47] This is now hailed as a watershed event for Apple's success[48] and was called a "masterpiece" by CNN[49] and one of the greatest TV advertisements of all time by TV Guide.[50][51]

The advertisement created great interest in the original Macintosh, and sales were initially good, but began to taper off dramatically after the first three months as reviews started to come in. Jobs had made the decision to equip the original Macintosh with 128 kilobytes of RAM, attempting to reach a US$1,000 (equivalent to $2,608 in 2021) price point, which limited its speed and the software that could be used. The Macintosh would eventually ship for US$2,495 (equivalent to $6,508 in 2021), a price panned by critics in light of its slow performance.[52][53][54][55]: 195  In early 1985, this sales slump triggered a power struggle between Steve Jobs and CEO John Sculley, who had been hired away from Pepsi two years earlier by Jobs[56][57] using the famous line, "Do you want to sell sugar water for the rest of your life or come with me and change the world?"[58] Sculley decided to remove Jobs as the head of the Macintosh division, with unanimous support from the Apple board of directors.[59][56]

The board of directors instructed Sculley to contain Jobs and his ability to launch expensive forays into untested products. Rather than submit to Sculley's direction, Jobs attempted to oust him from his leadership role at Apple.[60] Informed by Jean-Louis Gassée, Sculley found out that Jobs had been attempting to organize a boardroom coup and called an emergency meeting at which Apple's executive staff sided with Sculley and stripped Jobs of all operational duties.[60] Jobs resigned from Apple in September 1985 and took a number of Apple employees with him to found NeXT.[61] Wozniak had also quit his active employment at Apple earlier in 1985 to pursue other ventures, expressing his frustration with Apple's treatment of the Apple II division and stating that the company had "been going in the wrong direction for the last five years".[45][62][63] Despite Wozniak's grievances, he officially remained employed by Apple, and to this day continues to work for the company as a representative,[62] receiving a stipend estimated to be $120,000 per year for this role.[21] Both Jobs and Wozniak remained Apple shareholders after their departures.[64]

After the departures of Jobs and Wozniak, Sculley worked to improve the Macintosh in 1985 by quadrupling the RAM and introducing the LaserWriter, the first reasonably priced PostScript laser printer. PageMaker, an early desktop publishing application taking advantage of the PostScript language, was also released by Aldus Corporation in July 1985.[65] It has been suggested that the combination of Macintosh, LaserWriter and PageMaker was responsible for the creation of the desktop publishing market.[66]

This dominant position in the desktop publishing market[67] allowed the company to focus on higher price points, the so-called "high-right policy" named for the position on a chart of price vs. profits. Newer models selling at higher price points offered higher profit margin, and appeared to have no effect on total sales as power users snapped up every increase in speed. Although some worried about pricing themselves out of the market, the high-right policy was in full force by the mid-1980s, notably due to Jean-Louis Gassée's mantra of "fifty-five or die", referring to the 55% profit margins of the Macintosh II.[68]: 79–80 

This policy began to backfire in the last years of the decade as desktop publishing programs appeared on PC clones that offered some or much of the same functionality of the Macintosh, but at far lower price points. The company lost its dominant position in the desktop publishing market and estranged many of its original consumer customer base who could no longer afford their high-priced products. The Christmas season of 1989 was the first in the company's history to have declining sales, which led to a 20% drop in Apple's stock price.[68]: 117–129  During this period, the relationship between Sculley and Gassée deteriorated, leading Sculley to effectively demote Gassée in January 1990 by appointing Michael Spindler as the chief operating officer.[69] Gassée left the company later that year.[70]

"""

In [ ]:
len(apple_fruit_wiki.split(".")), len(apple_company_wiki.split("."))
apple_fruit_wiki = [x for x in apple_fruit_wiki.split(".") if len(x.split(" "))>5]
apple_company_wiki = [x for x in apple_company_wiki.split(".") if len(x.split(" "))>5]
wiki_sentences = np.asarray(apple_fruit_wiki + apple_company_wiki)
wiki_label_names = np.asarray(["apple_fruit"]*len(apple_fruit_wiki) + ["apple_company"]*len(apple_company_wiki))
wiki_labels = np.asarray([0]*len(apple_fruit_wiki) + [1]*len(apple_company_wiki))
class_names =  ['apple_fruit', 'apple_company']

In [ ]:
train, test, train_labels, test_labels = sklearn.model_selection.train_test_split(wiki_sentences, wiki_labels, test_size=.2, random_state=42)
train, val, train_labels, val_labels = sklearn.model_selection.train_test_split(train, train_labels, test_size=.1, random_state=42)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
val_labels = np.array(val_labels)

## Using Logistic Regression to classify Sentences 

In [ ]:
vectorizer = CountVectorizer(min_df=1)
vectorizer.fit(train)
train_vectors = vectorizer.transform(train)
test_vectors = vectorizer.transform(test)
val_vectors = vectorizer.transform(val)

In [ ]:
c = sklearn.linear_model.LogisticRegression()
# c = sklearn.ensemble.RandomForestClassifier(n_estimators=500, n_jobs=10)
c.fit(train_vectors, train_labels)
preds = c.predict(val_vectors)
print('Val accuracy', sklearn.metrics.accuracy_score(val_labels, preds))
def predict_lr(texts):
    return c.predict(vectorizer.transform(texts))

Val accuracy 0.9166666666666666


### Using UNK tokens to explain model prediction in Anchor

Perturbing examples by replacing words in a sentence with UNKS

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
explainer = anchor_text.AnchorText(nlp, class_names, use_unk_distribution=True)

In [ ]:
np.random.seed(1)
text = str(val[11])
pred = explainer.class_names[predict_lr([text])[0]]
alternative =  explainer.class_names[1 - predict_lr([text])[0]]
print('Prediction: %s' % pred)
exp = explainer.explain_instance(text, predict_lr, threshold=0.95)

Prediction: apple_fruit


In [ ]:
print('original sentence:')
print(text)

print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_different_prediction=True)]))

original sentence:
 Recently, double- and trihaploid individuals have been sequenced, yielding whole genome sequences of higher quality
Anchor: have AND genome
Precision: 0.99

Examples where anchor applies and model predicts apple_fruit:

UNK Recently UNK double- and UNK UNK have been sequenced UNK yielding whole genome sequences of UNK UNK
UNK UNK , UNK and UNK UNK have been UNK UNK yielding whole genome sequences of higher UNK
UNK Recently UNK UNK and trihaploid individuals have been sequenced UNK UNK UNK genome sequences UNK UNK quality
UNK UNK , UNK and UNK individuals have been sequenced , UNK whole genome UNK UNK UNK quality
UNK Recently UNK UNK UNK UNK individuals have been sequenced UNK UNK UNK genome sequences UNK higher UNK
  UNK , double- UNK trihaploid individuals have been sequenced , UNK UNK genome sequences UNK UNK quality
  UNK UNK UNK and trihaploid individuals have UNK sequenced , yielding whole genome UNK of higher quality
  UNK , UNK UNK UNK UNK have UNK sequenced 

In [ ]:
exp.show_in_notebook()

### Using BERT for text perturbation instead of UNK token

Above approach gives sentences that don't look realistic.

How about using BERT to perturb the data?

In [ ]:

explainer = anchor_text.AnchorText(nlp, class_names, use_unk_distribution=False)

In [ ]:
np.random.seed(1)
text = str(test[11])
pred = explainer.class_names[predict_lr([text])[0]]
alternative =  explainer.class_names[1 - predict_lr([text])[0]]
print('Prediction: %s' % pred)
start_time = time.time()
exp = explainer.explain_instance(text, predict_lr, threshold=0.95, verbose=True, onepass=False)
print('Time: %s' % (time.time() - start_time))

In [ ]:

print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(only_different_prediction=True)]))

Note: Here we are sampling from BERT sequentially (i.e. one mask at a time), to get sentences that are more coherent.

For making it faster, you can set onepass=True.

This will perform a single BERT pass per sample. You may lose some coherence, but it will be much faster. :)

In [ ]:
exp.show_in_notebook()